In [ ]:
%matplotlib inline

# Calibration scenario with a mesh-based output.


In [ ]:
from __future__ import annotations

from gemseo.algos.parameter_space import ParameterSpace
from gemseo.core.discipline import MDODiscipline
from numpy import array
from numpy import linspace

from gemseo_calibration.scenario import CalibrationMeasure
from gemseo_calibration.scenario import CalibrationScenario

Let us consider a function $f(x)=[ax,\gamma bx, \gamma]$
from $\mathbb{R}$ to $\mathbb{R}^11$
where $\gamma=[0,0.25,0.5,0.75,1.]$ plays the role of a mesh.
In practice,
we could imagine a model having an output related to a mesh $\gamma$
whose size and nodes would depend on the model inputs.
Thus, this mesh is also an output of the model.



In [ ]:
class Model(MDODiscipline):
    def __init__(self) -> None:
        super().__init__()
        self.input_grammar.update_from_names(["x", "a", "b"])
        self.output_grammar.update_from_names(["y", "z", "mesh"])
        self.default_inputs = {"x": array([0.0]), "a": array([0.0]), "b": array([0.0])}

    def _run(self) -> None:
        x_input = self.local_data["x"]
        a_parameter = self.local_data["a"]
        b_parameter = self.local_data["b"]
        y_output = a_parameter * x_input
        z_mesh = linspace(0, 1, 5)
        z_output = b_parameter * x_input[0] * z_mesh
        self.store_local_data(y=y_output, z=z_output, mesh=z_mesh)

This is a model of our reference data source,
which a kind of oracle providing input-output data
without the mathematical relationship behind it:



In [ ]:
class ReferenceModel(MDODiscipline):
    def __init__(self) -> None:
        super().__init__()
        self.input_grammar.update_from_names(["x"])
        self.output_grammar.update_from_names(["y", "z", "mesh"])
        self.default_inputs = {"x": array([0.0])}

    def _run(self) -> None:
        x_input = self.local_data["x"]
        y_output = 2 * x_input
        z_mesh = linspace(0, 1, 5)
        z_output = 3 * x_input[0] * z_mesh
        self.store_local_data(y=y_output, z=z_output, mesh=z_mesh)

However in this pedagogical example,
the mathematical relationship is known and we can see that
the parameters $a$ and $b$ must be equal to 2 and 3 respectively
so that the model and the reference are identical.

In the following,
we will try to find these values from several information sources.



Firstly,
we have a prior information about the parameters, that is $[a,b]\in[0,10]^2$:



In [ ]:
prior = ParameterSpace()
prior.add_variable("a", lower_bound=0.0, upper_bound=10.0, value=0.0)
prior.add_variable("b", lower_bound=0.0, upper_bound=10.0, value=0.0)

Secondly,
we have reference output data over the input space $[0.,3.]$:



In [ ]:
reference = ReferenceModel()
reference.set_cache_policy(reference.CacheType.MEMORY_FULL)
reference.execute({"x": array([1.0])})
reference.execute({"x": array([2.0])})
reference_data = reference.cache.to_dataset().to_dict_of_arrays(False)

From these information sources,
we can build and execute a
[CalibrationScenario][gemseo_calibration.scenario.CalibrationScenario]
to find the values of the parameters $a$ and $b$
which minimizes a
[CalibrationMeasure][gemseo_calibration.measure.CalibrationMeasure]
taking into account the outputs $y$ and $z$:



In [ ]:
model = Model()
control_outputs = [
    CalibrationMeasure("y", "MSE"),
    CalibrationMeasure("z", "ISE", "mesh"),
]
calibration = CalibrationScenario(model, "x", control_outputs, prior)
calibration.execute({
    "algo": "NLOPT_COBYLA",
    "reference_data": reference_data,
    "max_iter": 100,
})

Lastly,
we get the calibrated parameters:



plot an optimization history view:



In [ ]:
calibration.post_process("OptHistoryView", save=False, show=True)

as well as the model data versus the reference ones,
before and after the calibration:



In [ ]:
calibration.post_process("DataVersusModel", output="y", save=False, show=True)